In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

df = pd.read_parquet("../data/processed/airquality_with_anomalies.parquet")
df.head()

,CO_GT,PT08_S1_CO,NMHC_GT,C6H6_GT,PT08_S2_NMHC,NOx_GT,PT08_S3_NOx,NO2_GT,PT08_S4_NO2,PT08_S5_O3,Temp,Humidity,Absolute_Humidity,CO_STL_Residual,Z_Anomaly,IF_Anomaly,Final_Anomaly
Datetime,,,,,,,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,-0.216784,False,False,False
2004-03-10 19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,-0.025548,False,False,False
2004-03-10 20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,-3.961025,False,False,False
2004-03-10 21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,11.0,60.0,0.7867,-0.760440,False,False,False
2004-03-10 22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,11.2,59.6,0.7888,0.058204,False,False,False


In [3]:
clean_df = df[~df['Final_Anomaly']]
ts_clean = clean_df['CO_GT'].dropna()

In [4]:
from prophet import Prophet

prophet_df = ts_clean.reset_index()
prophet_df.columns = ['ds', 'y']

train_size = int(len(prophet_df) * 0.8)
train = prophet_df.iloc[:train_size]
test = prophet_df.iloc[train_size:]

model = Prophet()
model.fit(train)

future = model.make_future_dataframe(periods=len(test), freq='H')
forecast = model.predict(future)
yhat = forecast['yhat'].iloc[-len(test):]

21:43:37 - cmdstanpy - INFO - Chain [1] start processing
21:43:38 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Aishik Das\AppData\Local\Programs\Python\Python310\lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse_clean = np.sqrt(mean_squared_error(test['y'], yhat))
mae_clean = mean_absolute_error(test['y'], yhat)

print("Prophet RMSE (after cleaning):", rmse_clean)
print("Prophet MAE (after cleaning):", mae_clean)

Prophet RMSE (after cleaning): 31.75087885684458
Prophet MAE (after cleaning): 14.582834363473173


### Hence proved, removing anomalies improves forecasting stability and reduces extreme prediction errors.